In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from nodepy import rk, ivp
from sympy import Matrix, cos, sin, symbols, det, simplify, trace
from ipywidgets import interact

We first look at an example in which $A$ is constant (but non-normal), with negative real eigenvalues.

In [ ]:
omega = 1.
theta = np.pi/4
d1 = -1.
d2 = -10.

R = np.array([[ np.cos(omega-theta),-np.cos(omega)],
              [-np.sin(omega-theta), np.sin(omega)]])
D = np.diag([d1,d2])
Rinv = np.linalg.inv(R)
A = np.dot(R,np.dot(D,Rinv))
print("Eigenvalues: ", np.linalg.eigvals(A))

def rhs(t,u):
    return np.dot(A,u)

In [ ]:
u0 = np.array([1.,1.])
rhs(0.5,u0)
myivp = ivp.IVP(f=rhs,T=20.,u0=u0)

bs5 = rk.loadRKM('BS5')

t,y = bs5(myivp)

plt.plot(t,y);
plt.xlabel('$t$'); plt.ylabel('$y(t)$')
t = np.array(t);

Notice that the components of $y$ can grow initially even though the eigenvalues
of $A$ are negative.  This is due to the non-normality of $A$.  In fact, $y$ is
monotonically decaying **when expressed in the basis of the eigenvectors of $A$**
but not in the traditional basis.  We can see this by defining
$$
    Rz = y
$$
and looking at the behavior of the components of $z$.

In [ ]:
z = [Rinv@yy for yy in y]
plt.plot(t,z)
plt.plot(t,np.exp(-1*t)*z[0][0],'--k',alpha=0.5)
plt.plot(t,np.exp(-10*t)*z[0][1],'--',alpha=0.5);
plt.xlabel('$t$'); plt.ylabel('$z(t)$');

We can also understand this behavior by plotting the solution $y$ along
with its decomposition in the basis of the eigenvectors $R$.  Notice how
contraction of the second component (the green vector) actually moves the
solution away from the origin initially.

In [ ]:
def plot_frame(i=0):
    plt.plot(y[i][0],y[i][1],'ok')
    plt.plot([-2,2],[0,0],'--k')
    plt.plot([0,0],[-2,2],'--k')
    plt.xlim(-2,2);
    plt.ylim(-2,2);
    x1 = z[i][0]*R[0,0]
    x2 = z[i][0]*R[1,0]
    plt.plot([0,x1],[0,x2],'-b')
    plt.plot([x1,x1+z[i][1]*R[0,1]],[x2,x2+z[i][1]*R[1,1]],'-g')

interact(plot_frame,i=(0,len(y),1));

Now let's look at the case of time-dependent $A$.  We set it up so that the eigenvalues
are time-independent and the same as before.

In [ ]:
omega = 1.
theta = np.pi/4
d1 = -1.
d2 = -10.

def rhs(t,u):
    R = np.array([[ np.cos(omega*t-theta),-np.cos(omega*t)],
                  [-np.sin(omega*t-theta), np.sin(omega*t)]])
    D = np.diag([d1,d2])
    Rinv = np.linalg.inv(R)
    A = np.dot(R,np.dot(D,Rinv))
    return np.dot(A,u)

In [ ]:
omeg, thet, tt = symbols("omeg, thet, tt")
Rs = Matrix([[ cos(omeg*tt-thet), -cos(omeg*tt)],
            [-sin(omeg*tt-thet),  sin(omeg*tt)]])
Ds = Matrix([[d1, 0],
             [0, d2]])
M = Rs@Ds@Rs.inv()
print(M.eigenvals())

In [ ]:
u0 = np.array([1.,1.])
rhs(0.5,u0)
myivp = ivp.IVP(f=rhs,T=20.,u0=u0)

bs5 = rk.loadRKM('BS5')

t,y = bs5(myivp)

plt.plot(t,y);
t = np.array(t);
plt.title("Lambert's example");

For this particular case we still see decay, but at a much slower rate than
expected and with some additional oscillation.

# Vinograd-Dekker-Verwer example.

Finally, let's look at a case where the solution blows up even though the eigenvalues of $A(t)$
are the same as before.  This is the case presented by Dekker & Verwer, taken originally from Vinograd.

In [ ]:
omega = 1.
theta = np.pi/4
d1 = 1.j
d2 = -1.j
#d1 = -1
#d2 = -10
#omega = 6.
#theta = np.arctan(3/4.)

def rhs(t,u):
    R = np.array([[np.cos(omega*t-theta),-np.cos(omega*t)],
                  [-np.sin(omega*t-theta),np.sin(omega*t)]])
    D = np.diag([d1,d2])
    Rinv = np.linalg.inv(R)
    A = np.dot(R,np.dot(D,Rinv))
    return np.dot(A,u)

In [ ]:
u0 = np.array([0.,1.],dtype='complex64')
rhs(0.5,u0)
myivp = ivp.IVP(f=rhs,T=5.,u0=u0)

t,y = bs5(myivp)

ynorm = [np.linalg.norm(yi) for yi in y]

plt.plot(t,ynorm);
plt.title("Norm of $y(t)$");

We can calculate the expected rate of growth based on the eigenvalues of another matrix:

In [ ]:
M = np.array([[d1+omega/np.tan(theta), -omega/np.sin(theta)],
              [omega/np.sin(theta), d2-omega/np.tan(theta)]]);
rate = np.max(np.real(np.linalg.eigvals(M)))
rate

In [ ]:
plt.plot(t,ynorm)
plt.plot(t,np.exp(rate*np.array(t)))
plt.legend([r'$\|y(t)\|$',r'$\exp(\alpha t)$']);

In [ ]:
lim = 5
def plot_frame(i=0):
    tt = t[i]
    R = np.array([[np.cos(omega*tt-theta),-np.cos(omega*tt)],
                  [-np.sin(omega*tt-theta),np.sin(omega*tt)]])
    Rinv = np.linalg.inv(R)
    z = Rinv@y[i]
    plt.plot(y[i][0],y[i][1],'ok')
    plt.plot([-lim,lim],[0,0],'--k')
    plt.plot([0,0],[-lim,lim],'--k')
    plt.xlim(-lim,lim);
    plt.ylim(-lim,lim);
    x1 = z[0]*R[0,0]
    x2 = z[0]*R[1,0]
    plt.plot([0,x1],[0,x2],'-b')
    plt.plot([x1,x1+z[1]*R[0,1]],[x2,x2+z[1]*R[1,1]],'-g')

interact(plot_frame,i=(0,len(y)-1,10));

I thought that this plot would reveal what is going on, but so far
I don't understand it.